In [5]:
import sys 
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import copy

sys.path.insert(1, "/home/paperspace/devinterp") # TODO fix path

from devinterp.slt.sampler import Sampler, SamplerConfig, estimate_rlct
from torch.utils.data import TensorDataset
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [6]:
def train_one_epoch(model, train_loader, optimizer, criterion):
    model.train()
    train_loss = 0
    for data, target in tqdm(train_loader):
        optimizer.zero_grad()
        output = model(data.to(DEVICE))
        loss = criterion(output, target.to(DEVICE))
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
    return train_loss / len(train_loader)
def train_one_batch(model, train_loader, optimizer, criterion):
    model.train()
    train_loss = 0
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data.to(DEVICE))
        loss = criterion(output, target.to(DEVICE))
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        break
    return train_loss
def evaluate(model, test_loader, criterion):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data.to(DEVICE))
            loss = criterion(output, target.to(DEVICE))
            test_loss += loss.item()
    model.train()
    return test_loss / len(test_loader)


In [7]:

class PolyModel(nn.Module):
    def __init__(self, powers):
        super(PolyModel, self).__init__()
        self.weights = nn.Parameter(torch.tensor([1., 0.3],dtype=torch.float32, requires_grad=True))
        self.powers = powers
    def forward(self, x):
        multiplied = torch.prod(self.weights**self.powers)
        x = x*multiplied
        return x

# criterion = nn.MSELoss()
powers = torch.tensor([1, 2])
model = PolyModel(powers)
sigma = 0.5
lr = 0.0001
num_steps = 2000
num_train_samples = 5000
num_test_samples = 1000
batch_size = num_train_samples
w_true = torch.zeros_like(powers)

x = torch.normal(0, 2, size=(num_train_samples,))
y = sigma * torch.normal(0, 1, size=(num_train_samples,))
train_data = TensorDataset(x, y)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
# x_test = torch.normal(0, 2, size=(num_test_samples,))
# y_test =  sigma * torch.normal(0, 1, size=(num_test_samples,))
# test_data = TensorDataset(x_test, y_test)
# test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
train_losses = []
test_losses = []
rlct_estimates = []
models = []
test_loss = 0.
n_epochs = 20

def print_rlcts(SGNHT_config, SGLD_config):
    rlct_estimates_sgnht = []
    rlct_estimates_sgdl = []
    for epoch in range(n_epochs):
        # loss = train_one_epoch(model, train_loader, optimizer, criterion)
        # test_loss = evaluate(model, test_loader, criterion)
        # print(f"Epoch {epoch+1}, Train Loss: {loss}, Test Loss: {test_loss}")
        # print(model.state_dict()['weights'])
        model_for_rlct= copy.deepcopy(model)
        model_for_rlct2= copy.deepcopy(model)
        sgnht_sampler = Sampler(model_for_rlct, train_data, SGNHT_config)
        sgdl_sampler = Sampler(model_for_rlct2, train_data, SGLD_config)
        rlct_estimate_sgnht = sgnht_sampler.sample(summary_fn=estimate_rlct)
        rlct_estimate_sgdl = sgdl_sampler.sample(summary_fn=estimate_rlct)
        rlct_estimates_sgnht += [rlct_estimate_sgnht]
        rlct_estimates_sgdl += [rlct_estimate_sgdl]
        print(rlct_estimate_sgnht, rlct_estimate_sgdl)
        # raise Exception
    plt.hist(rlct_estimates_sgnht,alpha = 0.5, label='sgnht')
    plt.hist(rlct_estimates_sgdl,alpha = 0.5, label='sgdl')
    plt.legend()
    plt.show()



In [14]:
# Initialize sgnht sampler
SGNHT_config = SamplerConfig(
    optimizer_config=dict(
        optimizer_type="SGNHT",
        lr=lr,
        diffusion_factor=0.01,
        bounding_box_size=1.,
        num_samples=len(train_data),
        batch_size = batch_size,
    ),
    num_chains=1,
    num_draws_per_chain=10_000,
    num_burnin_steps=0,
    num_steps_bw_draws=1,
    batch_size=batch_size,         
    criterion = 'mse_loss' 
)
SGLD_config = SamplerConfig(
    optimizer_config=dict(
        optimizer_type="SGLD",
        lr=5*lr,
        noise_level=.9,
        # weight_decay=0.,
        elasticity=1,
        temperature='adaptive',
        num_samples=len(train_data),
    ),
    num_chains=1,
    num_draws_per_chain=10_000,
    num_burnin_steps=0,
    num_steps_bw_draws=1,
    verbose=False,
    batch_size=batch_size,         
    criterion = 'mse_loss'
)
print_rlcts(SGNHT_config, SGLD_config)

0.20992718636989594 0.015220989473164082
0.28192073106765747 0.009325043298304081
0.22444836795330048 0.01077716052532196
0.23088666796684265 0.04624031484127045
0.22681023180484772 0.0857624039053917
0.23961687088012695 0.006228358950465918
0.24253860116004944 0.01677807979285717
0.24888941645622253 0.03492080047726631
0.21274393796920776 0.034745845943689346
0.22688022255897522 0.11567951738834381
0.23842717707157135 0.04027438908815384
0.24453306198120117 0.016025777906179428
0.22892718017101288 0.03980201110243797
0.23944191634655 0.08833422511816025
0.25021907687187195 0.027992624789476395
